# Parâmetros iniciais

In [1]:
import json
import estrutura_ud
import os
import requests
from tqdm.autonotebook import tqdm
import pandas as pd

/tmp/ipykernel_27934/3486485871.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
dataset_file_template = 'test_{quality}_{dataset}'
gpt_file_template = '{quality}_sentences_{dataset}'

datasets = ['museu', 'porsimplessent']

dataset_qualities = ['complex', 'simple']
gpt_qualities = ['best', 'worst']

def load_file(f, extension):
    if extension == "txt":
        return f.read().splitlines()
    elif extension == "json":
        return json.load(f)

def load_files(extension):
    corpora = {}

    for dataset in datasets:
        for quality in dataset_qualities:
            dataset_file = dataset_file_template.format(quality=quality, dataset=dataset)
            with open(f"corpora/{dataset_file}.{extension}", 'r') as f:
                corpora[dataset_file] = load_file(f, extension)
        for quality in gpt_qualities:
            gpt_file = gpt_file_template.format(quality=quality, dataset=dataset)
            with open(f"corpora/{gpt_file}.{extension}", 'r') as f:
                corpora[gpt_file] = load_file(f, extension)

    return corpora


# Anotando os .txts com UDPipe

In [3]:
txt_files = load_files('txt')
model = "portuguese-petrogold-ud-2.12-230717"

corpora = {}
for txt_name, lines in tqdm(txt_files.items()):
    json_name = f"corpora/{txt_name}.json"
    if os.path.exists(json_name):
        continue

    corpora[txt_name] = []

    for line in tqdm(lines, leave=False):
        data = {
            'tokenizer': '',
            'tagger': '',
            'parser': '',
            'data': line,
            'model': model,
        }

        response = json.loads(requests.post('https://lindat.mff.cuni.cz/services/udpipe/api/process', data=data).content)['result']
        corpora[txt_name].append(response)
    
    with open(json_name, 'w') as f:
        f.write(json.dumps(corpora[txt_name]))

In [4]:
model = "portuguese-petrogold-ud-2.12-230717"

corpora = {}
for txt_name, lines in tqdm(txt_files.items()):
    json_name = f"corpora/{txt_name}.json"
    if os.path.exists(json_name):
        continue

    corpora[txt_name] = []

    for line in tqdm(lines, leave=False):
        data = {
            'tokenizer': '',
            'tagger': '',
            'parser': '',
            'data': line,
            'model': model,
        }

        response = json.loads(requests.post('https://lindat.mff.cuni.cz/services/udpipe/api/process', data=data).content)['result']
        corpora[txt_name].append(response)
    
    with open(json_name, 'w') as f:
        f.write(json.dumps(corpora[txt_name]))

# Carregando anotações CoNLL-U

In [30]:
corpora = load_files('json')
conllu = {}

for txt_name, lines in corpora.items():
    conllu[txt_name] = []
    for line in lines:
        corpus = estrutura_ud.Corpus()
        corpus.build(line)
        conllu[txt_name].append(corpus)

build: 0.00028204917907714844build: 0.00034880638122558594build: 0.001035928726196289build: 0.00064849853515625build: 0.0004107952117919922build: 0.0007309913635253906build: 0.00079345703125build: 0.0006077289581298828build: 0.0002961158752441406build: 0.0011205673217773438build: 0.00037407875061035156build: 0.0008211135864257812build: 0.00034332275390625build: 0.0007946491241455078build: 0.0005426406860351562build: 0.0006711483001708984build: 0.0002830028533935547build: 0.0005753040313720703build: 0.0008578300476074219build: 0.0009062290191650391build: 0.0007407665252685547build: 0.0007028579711914062build: 0.0018372535705566406build: 0.0004799365997314453build: 0.00027751922607421875build: 0.0003418922424316406build: 0.0006992816925048828build: 0.0006232261657714844build: 0.0004439353942871094build: 0.0005667209625244141build: 0.000865936279296875build: 0.0008487701416015625build: 0.0007905960083007812build: 0.0005104541778564453build: 0.00032067298889160156build: 0.00064778327941894

In [61]:
stats_idx = '''
1: Número de tokens por frase
2: Type/Token Ratio (TTR)
3: Lemma/Token Ratio (LTR)
4: Proporção de vírgulas por token
5: Proporção de orações por frase
6: Proporção de frases por entrada

7: Proporção de verbos para substantivos
8: Proporção de adjetivos para substantivos
9: Proporção de advérbios para verbos

10: Proporção de sujeitos pospostos para antepostos
11: Proporção de voz passiva para ativa
12: Proporção de locuções verbais (‘xcomp’ verbal) para verbos simples

13: Proporção de orações adverbiais
14: Proporção de orações adverbiais à esquerda do governante para à direita
15: Proporção de orações relativas desenvolvidas para reduzidas
18: Proporção de orações substantivas objetivas

16: Proporção de coordenações de orações
17: Proporção de coordenações de nominais
'''

stats_idx = {int(stat.split(':')[0].strip()): stat.split(':')[1].strip() for stat in stats_idx.splitlines() if stat.strip()}
stats = {stat: {} for stat in stats_idx.values()}

for txt_name in conllu:
    list_corpus = conllu[txt_name]

    n_tokens = 0
    n_sentences = 0
    n_commas = 0
    n_clauses = 0

    n_upos = {}
    n_deprel = {}
    n_passive = 0
    n_active = 0

    n_nsubj_left = 0
    n_nsubj_right = 0
    n_xcompverbal = 0

    n_advcl_left = 0
    n_advcl_right = 0

    n_conj_verb = 0
    n_conj_nominal = 0

    n_lines = len(list_corpus)
    n_multiplesentences = len([x for x in list_corpus if len(x.sentences) > 1])
    
    types = set()
    lemmas = set()

    for corpus in list_corpus:
        for sentid, sentence in corpus.sentences.items():
            
            n_sentences += 1

            for token in sentence.tokens:
                if '-' in token.id:
                    continue

                n_tokens += 1
                types.add(token.word.lower())
                lemmas.add(token.lemma.lower())

                if token.lemma == ",":
                    n_commas += 1
                if (token.upos == "VERB") or (token.upos == "AUX" and token.head_token.upos != "VERB"):
                    n_clauses += 1

                if not token.upos in n_upos:
                    n_upos[token.upos] = 0
                n_upos[token.upos] += 1

                if not token.deprel in n_deprel:
                    n_deprel[token.deprel] = 0
                n_deprel[token.deprel] += 1

                if 'Voice=Pass' in token.feats:
                    n_passive += 1
                else:
                    n_active += 1

                if token.deprel == "nsubj":
                    if int(token.id) < int(token.head_token.id):
                        n_nsubj_left += 1
                    else:
                        n_nsubj_right += 1

                if token.deprel == "xcomp" and token.head_token.upos == "VERB":
                    n_xcompverbal += 1

                if token.deprel == "advcl":
                    if int(token.id) < int(token.head_token.id):
                        n_advcl_left += 1
                    else:
                        n_advcl_right += 1

                if token.deprel == "conj":
                    if token.head_token.upos == "VERB":
                        n_conj_verb += 1
                    else:
                        n_conj_nominal += 1

    stats[stats_idx[1]][txt_name] = n_tokens / n_sentences
    stats[stats_idx[2]][txt_name] = len(types) / n_tokens
    stats[stats_idx[3]][txt_name] = len(lemmas) / n_tokens
    stats[stats_idx[4]][txt_name] = n_commas / n_tokens
    stats[stats_idx[5]][txt_name] = n_clauses / n_sentences
    stats[stats_idx[6]][txt_name] = n_sentences / n_lines

    stats[stats_idx[7]][txt_name] = n_upos.get("VERB", 0) / n_upos.get("NOUN", 1)
    stats[stats_idx[8]][txt_name] = n_upos.get("ADJ", 0) / n_upos.get("NOUN", 1)
    stats[stats_idx[9]][txt_name] = n_upos.get("ADV", 0) / n_upos.get("VERB", 1)

    stats[stats_idx[10]][txt_name] = n_nsubj_right / n_nsubj_left
    stats[stats_idx[11]][txt_name] = n_passive / n_active
    stats[stats_idx[12]][txt_name] = n_xcompverbal / (n_upos.get("VERB") - n_xcompverbal)
    stats[stats_idx[13]][txt_name] = n_deprel.get("advcl", 0) / n_clauses

    stats[stats_idx[14]][txt_name] = n_advcl_left / n_advcl_right
    stats[stats_idx[15]][txt_name] = n_deprel.get("acl:relcl", 0) / n_deprel.get("acl", 1)
    stats[stats_idx[18]][txt_name] = n_deprel.get("ccomp", 0) / n_clauses

    stats[stats_idx[16]][txt_name] = n_conj_verb / n_clauses
    stats[stats_idx[17]][txt_name] = n_conj_nominal / n_clauses

df = pd.DataFrame(stats)
df_museu = df[df.index.str.contains('museu')]
df_porssimplessent = df[df.index.str.contains('porsimplessent')]

df_museu

,Número de tokens por frase,Type/Token Ratio (TTR),Lemma/Token Ratio (LTR),Proporção de vírgulas por token,Proporção de orações por frase,Proporção de frases por entrada,Proporção de verbos para substantivos,Proporção de adjetivos para substantivos,Proporção de advérbios para verbos,Proporção de sujeitos pospostos para antepostos,Proporção de voz passiva para ativa,Proporção de locuções verbais (‘xcomp’ verbal) para verbos simples,Proporção de orações adverbiais,Proporção de orações adverbiais à esquerda do governante para à direita,Proporção de orações relativas desenvolvidas para reduzidas,Proporção de orações substantivas objetivas,Proporção de coordenações de orações,Proporção de coordenações de nominais
test_complex_museu,21.437751,0.190240,0.146684,0.051798,2.674699,1.046218,0.507660,0.291489,0.327745,0.031390,0.015794,0.114953,0.213964,0.325581,0.915385,0.030030,0.092342,0.153904
test_simple_museu,15.603399,0.172204,0.127723,0.044390,2.026912,1.483193,0.517850,0.222815,0.338352,0.038028,0.004926,0.107989,0.157932,0.537415,2.562500,0.045423,0.068484,0.149546
best_sentences_museu,18.704312,0.212647,0.159183,0.045669,2.494867,1.023109,0.528832,0.264662,0.273066,0.024291,0.011549,0.107327,0.201646,0.346154,1.337500,0.034568,0.107819,0.167901
worst_sentences_museu,18.253608,0.218344,0.164803,0.042359,2.498969,1.018908,0.538385,0.271685,0.259259,0.032333,0.017937,0.120332,0.212046,0.297980,1.087912,0.036304,0.092409,0.148515


In [62]:
df_porssimplessent

,Número de tokens por frase,Type/Token Ratio (TTR),Lemma/Token Ratio (LTR),Proporção de vírgulas por token,Proporção de orações por frase,Proporção de frases por entrada,Proporção de verbos para substantivos,Proporção de adjetivos para substantivos,Proporção de advérbios para verbos,Proporção de sujeitos pospostos para antepostos,Proporção de voz passiva para ativa,Proporção de locuções verbais (‘xcomp’ verbal) para verbos simples,Proporção de orações adverbiais,Proporção de orações adverbiais à esquerda do governante para à direita,Proporção de orações relativas desenvolvidas para reduzidas,Proporção de orações substantivas objetivas,Proporção de coordenações de orações,Proporção de coordenações de nominais
test_complex_porsimplessent,22.518868,0.282223,0.223782,0.053275,2.460692,1.049505,0.484141,0.259672,0.388049,0.074020,0.010157,0.152697,0.143131,0.493333,0.814607,0.063898,0.051118,0.146326
test_simple_porsimplessent,21.882445,0.262302,0.197980,0.041043,2.467085,1.052805,0.478485,0.232014,0.359712,0.058981,0.009180,0.159299,0.123253,0.259740,1.029240,0.071792,0.055909,0.140407
best_sentences_porsimplessent,20.227414,0.276067,0.215694,0.041275,2.334891,1.059406,0.497162,0.249716,0.339422,0.018767,0.011607,0.134715,0.150767,0.255556,0.992308,0.090060,0.054036,0.150767
worst_sentences_porsimplessent,19.194053,0.288626,0.227477,0.042968,2.287950,1.054455,0.526804,0.268440,0.322877,0.022727,0.011797,0.146491,0.162791,0.259259,0.934426,0.085499,0.044460,0.141587
